<h1>Kapitel 8 - Hypotestest för två populationer</h1>
Här kommer vi gå igenom hur man kan beräkna test-statistiskor, samt utföra test, för jämförelse mellan två populationer, med hjälp av olika paket i Python. Vi börjar med att importera lite standardpaket och ladda in vår favorit-data, Palmer Penguins.

In [1]:
#importera paket och exempeldata 
import numpy as np
import pandas as pd

filepath = 'c:/users/davber/Python/notebooks/data/penguins.csv'
penguins = pd.read_csv(filepath)
penguins = penguins.dropna() # Plocka bort rader som innehåller NaN

<h3>Hypotestest för varians mellan två populationer</h3>
<h5>Konstruera ett hypotestest som undersöker om variansen vad gäller näbblängd är större för han- än för honpingviner. Använd en signifikansnivå på $\alpha = 0.05$</h5>
Vi börjar med att formulera våra hypoteser. Här vill vi göra ett ensidigt test mellan två varianser $\sigma^2_1$ och $\sigma^2_2$. Vi anger vår alternativhypotes som att $\sigma^2_{hane}$ är större än, $\sigma^2_{hona}$ alltså:<br>
    $H_A: \sigma^2_{hane} > \sigma^2_{hona}$<br>
Det följer att nollhypotesen, alltså komplementet till alternativhypotesen då blir:<br>
    $H_0: \sigma^2_{hane} \leq \sigma^2_{hona}$
<br><br>
Vi delar upp vårt ursprungsdataset i två delar, en som innehåller alla hanpingviner och en som innehåller alla honpingviner.

In [2]:
male = penguins[penguins['sex'] == 'male']
female = penguins[penguins['sex'] == 'female']

n_male = len(male)
n_female = len(female)

Nästa steg är att beräkna testfunktionen, i fallet med två varianser är det enkelt. Den ges av:
<h4>
    $F=\frac{\sigma^2_1}{\sigma^2_2}$
</h4>

Vi kan alltså lätt räkna ut den med våra kända NumPy-metoder enligt följande:

In [3]:
statistic = np.var(male['bill_length_mm'], ddof=1) / np.var(female['bill_length_mm'], ddof=1)
print('F-statistic: ' + str(statistic))

F-statistic: 1.1979487109657414


Vi kan jämföra den med det kritiska värdet för vår F-fördelning (givet $\alpha = 0.05$) genom att hämta F-fördelningen i SciPy och kika i dess kumulativa fördelningsfunktion:

In [4]:
from scipy.stats import f

result = f.ppf(q=0.05, dfn=n_male-1, dfd=n_female-1)
print('F-critical :' +str(result))

F-critical :0.7739172769072281


Värdet på vår beräknade F-statistika (1.20) är högre än det kritiska värdet (0.77), och vi kan därmed förkasta nollhypotesen. Utifrån vårt test kan vi alltså konstatera att med ett konfidens på 95%, så skiljer sig variansen i näbblängd åt mellan han- och honpingviner.<br><br>

Här föreligger tyvärr samma problem som med två-sidiga varianstester från chi2-fördelningen, alltså att jag inte känner till någon distribuerad funktion som gör ett sådant här test åt en. Den som vill utmana sig kan försöka bygga en funktion som utför ett sådant här test! För något att utgå ifrån finns ett liknande program i notebook 7, som utför ett chi2-test för varians. 

<h3>Hypotestest för medelvärde mellan två populationer</h3>
<h5>Konstruera ett hypotestest som undersöker om medelvärdet i näbblängd är större för hanpingviner än för honpingviner. Använd en signifikansnivå $\alpha=0.05$. Antag $d_0 = 0$.</h5>

Det börjar kännas bekant det här nu va! Vi börjar med att formulera våra hypoteser. Här vill vi göra ett ensidigt test mellan två medelvärden $\mu_{hane}$ och $\mu_{hona}$. Vi anger vår alternativhypotes som att $\mu_{hane}$ är större än $\mu_{hona}$, alltså:<br>
    $H_A: \mu_{hane} > \mu_{hona}$<br>
Det följer att nollhypotesen, alltså komplementet till alternativhypotesen då blir:<br>
    $H_0: \mu_{hane} \leq \mu_{hona}$<br>
Testfunktionen i det här fallet, där båda varianserna är okända, samt får antas kunna vara olika, blir:
<h3>
    $t_{df} = \frac{\bar{x_1} - \bar{x_2}}{\sqrt{\frac{s_1^2}{n_1} + \frac{s_2^2}{n_2}}}$
</h3>
Här skulle vi kunna roa oss med att beräkna test-statistiskan med NumPy-metoder. Men, vi kan också nöja oss med att låta SciPy räkna på det åt oss, genom att gå direkt till funktionen för ett t-test mellan två medelvärden <code>ttest_ind()</code>. argumentet <code>equal_var</code> avgör här om testet skall göras utifrån antagande om att variansen är lika eller olika.

In [5]:
from scipy.stats import ttest_ind

result = ttest_ind(a=male['bill_length_mm'], b=female['bill_length_mm'], equal_var=False, alternative='greater')
print(result)

Ttest_indResult(statistic=6.67245551331115, pvalue=5.332283057920293e-11)


Vi får ett p-värde på ~5e-11, alltså bra nog att förkasta nollhypotesen till nära en mycket liten signifikansnivå. Vår data säger alltså att medellängden på pingvinnäbbar inte är samma för hanar och honor, till en signifikans på åtminstone 0.05.

Om vi istället jämför mellan Adelie-pingviner på Dream respektive Biscoe island då? Här kan vi kanske förvänta oss en betydligt mindre, om någon skillnad. Det fina med att göra sådana här beräkningar m.h.a. Pandas och SciPy är att det med vad vi nu lärt oss är väldigt enkelt att sätta upp det testet.

In [6]:
dream = penguins[(penguins['species'] == 'Adelie') & (penguins['island'] == 'Dream')]
biscoe = penguins[(penguins['species'] == 'Adelie') & (penguins['island'] == 'Biscoe')]

result = ttest_ind(a=dream['bill_length_mm'], b=biscoe['bill_length_mm'], equal_var=True, alternative='two-sided')
print(result)

Ttest_indResult(statistic=-0.9060705107779293, pvalue=0.36714374025851804)


Nej, här visar det sig att p-värdet blev ganska högt istället, ~0.37. Testet ger alltså att vi inte kan förkasta nollhypotesen, vi kan alltså utifrån vår data, och en signifikansnivå på 0.05, inte avgöra om det är någon skillnad mellan medelnäbblängden på Adelie-pingviner på Dream resp. Biscoe island.

Obs! Notera att vi här ställde <code>equal_var=True</code>. Det är ett rimligt antagande att Adeliepingviners näbblängd har samma varians oavsett vilken ö de befinner sig på. (För så stora <i>n</i> som vi jobbar med här spelar det nästan ingen roll)